# Project Description

#### Motivation  

This is my proposal for the open subject capstone project for conclusion of the **Data Engineering** Nanodegree at Udacity.  

For this project, I am looking to target a relevant subject, with real world application, and that produces large amounts of data frequently.  

One of the subject that immediately came to my mind is COVID-19. This subject can and has been explored in different ways, and in my view, it can be interesting to see the way travelling and economies were affected by it. This will still unfortunately going to be a relevant subject for months to come, rich in data, and that can be approached in multiple angles.     

In this exercise, I will explore some data sets and approaches to the subject. 


#### Scope  
TODO  

#### Data Exploration

In [1]:
import configparser
conf = configparser.ConfigParser()

In [2]:
import requests

In [3]:
conf.read("configs/aviation.cfg")

[]

##### Flight data  :

https://aviationstack.com/  

Historical data not supported. 500 Requests. 100 results per API request.  
Airline, airport, cities data.  

In [ ]:
aviation_key = conf["AVIATIONSTACK"]["ACCESS_KEY"]
aviation_historical_endpoint = 'http://api.aviationstack.com/v1/flights?access_key={access_key}'.format( 
    access_key = aviation_key 
)

In [ ]:
response = requests.get(aviation_historical_endpoint)

In [ ]:
realtime_flights = response.json()
realtime_flights

##### OpenSky  resources

https://traffic-viz.github.io/scenarios/covid19.html  
https://traffic-viz.github.io/opensky_rest.html

In [ ]:
#!conda install cartopy shapely

In [ ]:
#!pip install --upgrade traffic

In [ ]:
import traffic
## SET opensky_username and opensky_password in following file:
traffic.config_file

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

from traffic.data import opensky
from traffic.drawing import EuroPP, countries

/home/nunovazafonso/anaconda3/lib/python3.8/site-packages/traffic/data/__init__.py:65: DeprecationWarning: Please edit your configuration file:

        # Old style, will soon no longer be supported
        [global]
        opensky_username =
        opensky_password =

        # New style

        [opensky]
        username =
        password =

        
  warnings.warn(


### Current flights

In [ ]:
# States -> Current flights
sv = opensky.api_states()

with plt.style.context('traffic'):
    fig, ax = plt.subplots(subplot_kw=dict(projection=EuroPP()))
    ax.add_feature(countries())
    ax.gridlines()
    ax.set_extent((-7, 15, 40, 55))

    sv.plot(ax, s=10)

In [ ]:
# REST API to open sky 
opensky_endpoint = 'https://opensky-network.org/api'
arrivals_endpoint = '/flights/arrival'
departures_endpoint = '/flights/departure'
all_flights_endpoint = '/flights/all'

### Arrivals

In [ ]:
# set ARRIVALS params -> 7 days max
params = {
    'airport': 'EDDF'
    ,'begin': 1612183863
    ,'end': 1612615863
} 
arrivals_response = requests.get(opensky_endpoint + arrivals_endpoint , params )

In [ ]:
# Get demo ARRIVALS 
arrivals_response.json()

### All flights

In [ ]:
# set ALL FLIGHTS params -> two hours max
params = {
    'begin': 1612183863
    ,'end': 1612184863
} 
all_flights_response = requests.get(opensky_endpoint + all_flights_endpoint , params )

In [ ]:
# Get demo ALL FLIGHTS
all_flights_response.json()

### Loading CSV files

In [ ]:
#!pip install zenodo-get

In [ ]:
!cd input/data | zenodo_get 4485741

In [7]:
import pandas as pd

/home/nunovazafonso/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pd.read_csv('input/data/flightlist_20190101_20190131.csv.gz', 
            header=0, 
            sep=',', 
            quotechar='"', 
            error_bad_lines=False)

### Twitter data   

http://twitterdata.covid19dataresources.org/data/  
https://zenodo.org/record/4568860#.YDvUKv7LcYs  

In [ ]:
# Download Twitter data
!zenodo_get 4568860 -o input/data/twitter 

In [ ]:
pd.read_json('input/data/twitter/covid19_2020_02_06.json.gz' 
             , compression = 'gzip'
             , lines=True)

## Covid Data

https://github.com/owid/covid-19-data/tree/master/public/data/vaccinations  
https://data.humdata.org/dataset/coronavirus-covid-19-cases-and-deaths  

### Vaccination

In [8]:
vaccination_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.json'

In [9]:
daily_vaccinations = pd.json_normalize( requests.get(vaccination_url).json() 
                  ,meta=['country', 'iso_code']
                  ,record_path= 'data' )

daily_vaccinations[daily_vaccinations['country'] == 'Portugal'].sort_values('date')

,date,total_vaccinations,people_vaccinated,total_vaccinations_per_hundred,people_vaccinated_per_hundred,daily_vaccinations,daily_vaccinations_per_million,daily_vaccinations_raw,people_fully_vaccinated,people_fully_vaccinated_per_hundred,country,iso_code
5950,2020-12-27,4963.0,4963.0,0.05,0.05,NaN,NaN,NaN,NaN,NaN,Portugal,PRT
5951,2020-12-28,7754.0,7754.0,0.08,0.08,2791.0,274.0,2791.0,NaN,NaN,Portugal,PRT
5952,2020-12-29,20368.0,20368.0,0.20,0.20,7702.0,755.0,12614.0,NaN,NaN,Portugal,PRT
5953,2020-12-30,30543.0,30543.0,0.30,0.30,8527.0,836.0,10175.0,NaN,NaN,Portugal,PRT
5954,2020-12-31,32749.0,32749.0,0.32,0.32,6946.0,681.0,2206.0,NaN,NaN,Portugal,PRT
...,...,...,...,...,...,...,...,...,...,...,...,...
6029,2021-03-16,1193186.0,849464.0,11.70,8.33,21299.0,2089.0,24971.0,343722.0,3.37,Portugal,PRT
6030,2021-03-17,1200691.0,851022.0,11.78,8.35,20801.0,2040.0,7505.0,349669.0,3.43,Portugal,PRT
6031,2021-03-18,1235136.0,863570.0,12.11,8.47,22339.0,2191.0,34445.0,371566.0,3.64,Portugal,PRT
6032,2021-03-19,1280409.0,875490.0,12.56,8.59,23789.0,2333.0,45273.0,404919.0,3.97,Portugal,PRT


### Cases and Deaths

In [10]:
covid_url = 'https://covid19.who.int/WHO-COVID-19-global-data.csv'
# daily data on COVID cases and deaths
cases_df = pd.read_csv( covid_url )

cases_df[cases_df['Country'] == 'Portugal' ].sort_values('Date_reported')

/home/nunovazafonso/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
74592,2020-01-03,PT,Portugal,EURO,0,0,0,0
74593,2020-01-04,PT,Portugal,EURO,0,0,0,0
74594,2020-01-05,PT,Portugal,EURO,0,0,0,0
74595,2020-01-06,PT,Portugal,EURO,0,0,0,0
74596,2020-01-07,PT,Portugal,EURO,0,0,0,0
...,...,...,...,...,...,...,...,...
75031,2021-03-17,PT,Portugal,EURO,384,814897,13,16707
75032,2021-03-18,PT,Portugal,EURO,673,815570,15,16722
75033,2021-03-19,PT,Portugal,EURO,485,816055,21,16743
75034,2021-03-20,PT,Portugal,EURO,568,816623,11,16754


In [ ]:
destination_folder = "input/data" 
cases_df.to_csv( destination_folder + "/covid_data.csv", sep="," )


### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up